Solve linear systems using iterations methods (Book 1, p. 102, ex. 12.7).

In [1]:
import numpy as np
from numpy.linalg import solve, inv, norm, eig

import pandas as pd

# Common parts

In [2]:
# Initial test matrix.
A = np.array([[7.35272,  0.88255,  -2.270052],
              [0.88255,  5.58351,  0.528167],
              [-2.27005, 0.528167, 4.430329]])

b = np.array([[1.],
              [0.],
              [0.]])

# Gaussian elimination

In [3]:
def gaussian_elimination(A, b, eps=1e-5):
    """Solving linear system using gaussian elimination.

    Args:
        A (ndarray<ndarray, ndarray>): matrix of coefficents.
        b (ndarray): vector of values.
        eps (float): all values below eps equivalent to zero.

    Returns:
        x (ndarray): solution.
    """

    # Getting matrix shape.
    n = A.shape[0]

    # Merging coefficents with values.
    Ab = np.concatenate((A, b), axis=1)

    # Making upper triangular matrix.
    for k in range(0, n):

        # Dividing all row alements after
        # diagonal element on diagonal
        # element.
        tmp = Ab[k][k]
        if np.abs(tmp) < eps:
            print("\nElement Ab[{}][{}]={} smaller than eps={}.".format(
                k, k, tmp, eps))
        for j in range(k, n + 1):
            Ab[k][j] = Ab[k][j] / tmp

        # Substracting top element multiplied
        # by 1st element in row from each
        # element.
        for i in range(k + 1, n):
            tmp = Ab[i][k]
            for j in range(k, n + 1):
                Ab[i][j] = Ab[i][j] - Ab[k][j] * tmp

    # Solve equation for an upper
    # triangular matrix Ab.
    x = np.zeros((3, 1))
    for i in range(n - 1, -1, -1):
        x[i] = Ab[i][n] / Ab[i][i]
        for k in range(i - 1, -1, -1):
            Ab[k][n] -= Ab[k][i] * x[i]

    return x

In [4]:
gauss_solution = gaussian_elimination(A, b) 
print("\nGauss solution: \n{}".format(gauss_solution))


Gauss solution: 
[[ 0.16810435]
 [-0.03511502]
 [ 0.09032103]]


# Iteration method

In [5]:
def iterations_prepare(A, b, verbose=True):
    """prepares linear system to x = h_d * x + g_d form.

    Args:
        A (ndarray<ndarray, ndarray>): matrix of coefficents.
        b (ndarray): vector of values.

    D is diagonal matrix with coefficents from A diagonal. 

    Returns:
        H_D (ndarray<ndarray, ndarray>): E - D^(-1) * A.
        g_D (ndarray): D^(-1) * b.
    """

    verboseprint = print if verbose else lambda *a, **k: None

    verboseprint("\nA: \n{}".format(A))
    verboseprint("\nb: \n{}".format(b))

    D = np.diag(np.diag(A))
    verboseprint("\nD: \n{}".format(D))

    H_D = np.identity(A.shape[0]) - np.dot(inv(D), A)
    verboseprint("\nH_D: \n{}".format(H_D))

    g_D = np.dot(inv(D), b)
    verboseprint("\ng_D: \n{}".format(g_D))

    H_D_norm = norm(H_D, np.inf)
    verboseprint("\n||H_D||_inf: \n{}".format(H_D_norm))

    return H_D, g_D

In [6]:
def apriori_iterations_estimation(H, g, verbose=True, eps=1e-5, x_0=[[0.],
                                                                     [0.],
                                                                     [0.]]):
    """Apriori estimating number of iterations.

    Find such k, that difference between
    answer and solution will be less then eps.

    Args:
        H (ndarray<ndarray, ndarray>): E - D^(-1) * A.
        g (ndarray): D^(-1) * b.
        eps (float): solution error rate.
        x_0 (ndarray<ndarray, ndarray>): starting vector.

    Returns:
        k (int): estimated number of iterations.
    """

    verboseprint = print if verbose else lambda *a, **k: None

    k = 1
    while True:
        error = (norm(H, np.inf)**k) * norm(x_0, np.inf) + \
                (norm(H, np.inf)**k) * norm(g, np.inf) / \
                (1 - norm(H, np.inf))
        if (error < eps):
            break
        k += 1

    verboseprint("\nEstimated number of iterations: \n{}".format(k))

    return k

In [7]:
def iterations_method(A, b, k, verbose=True, x_0=[[0.],
                                                  [0.],
                                                  [0.]]):
    """Solving linear system using iterations method.

    Args:
        A (ndarray<ndarray, ndarray>): matrix of coefficents.
        b (ndarray): vector of values.
        k (int): number of iterations.
        x_0 (ndarray<ndarray, ndarray>): starting vector.

    Returns:
        x (ndarray): solution.
    """

    verboseprint = print if verbose else lambda *a, **k: None

    H, g = iterations_prepare(A, b, verbose=verbose)

    for i in range(k):
        x = np.dot(H, x_0) + g
        x_0 = x

    verboseprint("\nIterations solution: \n{}".format(x))

    return x

In [8]:
def iterations_method_stats(A, b, x_solution, eps=1e-5, verbose=True, x_0=[[0.],
                                                                           [0.],
                                                                           [0.]]):
    """Printing stats about iterations method.

    Args:
        A (ndarray<ndarray, ndarray>): matrix of coefficents.
        b (ndarray): vector of values.
        x_solution (ndarray): actual solution.
        eps (float): solution error rate.
        x_0 (ndarray<ndarray, ndarray>): starting vector.

    Returns:
        k (int): number of iterations to hit eps error.
    """

    verboseprint = print if verbose else lambda *a, **k: None

    H, g = iterations_prepare(A, b, verbose=False)

    k = 1
    x = x_0
    while norm(x_solution - x, np.inf) > eps:
        x_0 = x
        x = iterations_method(A, b, k, verbose=False)
        k += 1

    error = norm(x_solution - x, np.inf)
    aprior_error = (norm(H, np.inf)**k) * norm(x_0, np.inf) + \
                   (norm(H, np.inf)**k) * norm(g, np.inf) / \
                   (1 - norm(H, np.inf))
    prior_error = norm(H, np.inf) * norm(x - x_0, np.inf) / \
                  (1 - norm(H, np.inf))
    
    
    # Stats for lusternik solution.
    w, _ = eig(H)
    r = np.max(w)
    lusternik_solution = x_0 + (x - x_0) / (1 - r)
    lusternik_error = norm(x_solution - lusternik_solution, np.inf)
    
    verboseprint("\nSolution: \n{}".format(x))
    verboseprint("\nNumber of iterations to reach given error rate: \n{}".format(k))
    verboseprint("\nAposterior error: \n{}".format(aprior_error))
    verboseprint("\nPosterior error: \n{}".format(prior_error))
    verboseprint("\nError: \n{}".format(error))
    verboseprint("\nLusternik error: \n{}".format(lusternik_error))

    return [k, error, lusternik_error]

### Finding apriori iterations number

In [9]:
H, g = iterations_prepare(A, b)
k = apriori_iterations_estimation(H, g)


A: 
[[ 7.35272   0.88255  -2.270052]
 [ 0.88255   5.58351   0.528167]
 [-2.27005   0.528167  4.430329]]

b: 
[[1.]
 [0.]
 [0.]]

D: 
[[7.35272  0.       0.      ]
 [0.       5.58351  0.      ]
 [0.       0.       4.430329]]

H_D: 
[[ 0.         -0.12003041  0.30873636]
 [-0.15806366  0.         -0.09459408]
 [ 0.51238858 -0.1192162   0.        ]]

g_D: 
[[0.13600409]
 [0.        ]
 [0.        ]]

||H_D||_inf: 
0.6316047860102488

Estimated number of iterations: 
23


### Finding solution

In [10]:
iteration_result = iterations_method_stats(A, b, gauss_solution)


Solution: 
[[ 0.1680997 ]
 [-0.03511201]
 [ 0.09031434]]

Number of iterations to reach given error rate: 
14

Aposterior error: 
0.0008638344045932408

Posterior error: 
1.1857412019449837e-05

Error: 
6.682709230584893e-06

Lusternik error: 
1.6312125191431104e-06


# Seidel method

In [11]:
def seidel_method(A, b, k, verbose=True, x_0=[[0.],
                                              [0.],
                                              [0.]]):
    """Solving linear system using seidel method.

    Args:
        A (ndarray<ndarray, ndarray>): matrix of coefficents.
        b (ndarray): vector of values.
        k (int): number of iterations.
        x_0 (ndarray<ndarray, ndarray>): starting vector.

    Returns:
        x (ndarray): solution.
    """

    verboseprint = print if verbose else lambda *a, **k: None

    H, g = iterations_prepare(A, b, verbose=verbose)

    H_L = np.tril(H, -1)
    H_R = np.triu(H)
    
    verboseprint("\nH_L: \n{}".format(H_L))
    verboseprint("\nH_R: \n{}".format(H_R))
    
    identity = np.identity(A.shape[0])
    for i in range(k):
        x = np.dot(inv(identity - H_L), np.dot(H_R, x_0)) + \
            np.dot(inv(identity - H_L), g)
        x_0 = x

    verboseprint("\nSeidel method solution: \n{}".format(x))

    return x

In [12]:
def seidel_method_stats(A, b, x_solution, eps=1e-5, verbose=True, x_0=[[0.],
                                                                       [0.],
                                                                       [0.]]):
    """Printing stats about seidel method.

    Args:
        A (ndarray<ndarray, ndarray>): matrix of coefficents.
        b (ndarray): vector of values.
        x_solution (ndarray): actual solution.
        eps (float): solution error rate.
        x_0 (ndarray<ndarray, ndarray>): starting vector.

    Returns:
        k (int): number of iterations to hit eps error.
    """

    verboseprint = print if verbose else lambda *a, **k: None

    H, g = iterations_prepare(A, b, verbose=False)

    H_L = np.tril(H, -1)
    H_R = np.triu(H)
    
    identity = np.identity(A.shape[0])
    
    k = 1
    x = x_0
    while norm(x_solution - x, np.inf) > eps:
        x_0 = x
        x = seidel_method(A, b, k, verbose=False)
        k += 1


    error = norm(x_solution - x, np.inf)
    
    # Stats for lusternik solution.
    w, _ = eig(np.dot(inv(identity - H_L), H_R))
    r = np.max(w)
    lusternik_solution = x_0 + (x - x_0) / (1 - r)
    lusternik_error = norm(x_solution - lusternik_solution, np.inf)
    
    verboseprint("\nSolution: \n{}".format(x))
    verboseprint("\nNumber of iterations to reach given error rate: \n{}".format(k))
    verboseprint("\nError: \n{}".format(error))
    verboseprint("\nLusternik error: \n{}".format(lusternik_error))

    return [k, float(error), lusternik_error]

### Finding solution

In [13]:
siedel_result = seidel_method_stats(A, b, gauss_solution)


Solution: 
[[ 0.16810058]
 [-0.03511352]
 [ 0.09031892]]

Number of iterations to reach given error rate: 
8

Error: 
3.7638519359839417e-06

Lusternik error: 
2.5320162633235554e-12


# Successive over-relaxation method

# Methods comparison

In [14]:
pd.options.display.float_format = '{:.2e}'.format

columns = ['Method', 'Iterations', 'Error', 'Lusternik error']
results = pd.DataFrame(columns=columns)

results.loc[len(results)] = ['Iterations', *iteration_result]
results.loc[len(results)] = ['Siedel', *siedel_result]

results.set_index('Method', inplace=True)
results = results.astype({'Error': 'float', 'Lusternik error': 'float'})

In [15]:
display(results)

,Iterations,Error,Lusternik error
Method,,,
Iterations,14,6.68e-06,1.63e-06
Siedel,8,3.76e-06,2.53e-12
